# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

In [2]:
# Configure gmaps
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
df = pd.read_csv("weatherData.csv")
df.head()


,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,kutum,14.20,24.67,69.37,15,1009,3.76,15
1,provideniya,64.38,-173.30,38.68,63,1017,1.45,0
2,norman wells,65.28,-126.83,39.20,60,1023,16.11,75
3,mahon,39.89,4.27,62.60,88,1014,4.70,65
4,tuktoyaktuk,69.45,-133.04,17.60,78,1026,16.11,20


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
# Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = df["humidity"]

In [5]:
# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
mask1 = (df.temperature >= 70) & (df.temperature < 80)
mask2 = df.wind_speed < 10
mask3 = df.cloudiness == 0
mask4 = mask1 & mask2 & mask3

ideal_cities = df.loc[mask4].reset_index(drop=True)
ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,antalaha,-14.90,50.28,72.91,83,1016,6.53,0
1,sumter,28.71,-82.08,72.70,78,1016,1.99,0
2,rock sound,24.90,-76.20,75.83,65,1016,8.90,0
3,abu dhabi,24.47,54.37,78.64,73,1010,1.12,0
4,contamana,-7.33,-75.02,74.19,87,1010,1.12,0
5,sumbe,-11.21,13.84,77.95,82,1011,5.73,0
6,sao filipe,14.90,-24.50,74.61,73,1014,6.49,0
7,morondava,-20.28,44.28,72.97,67,1015,5.01,0
8,maragogi,-9.01,-35.22,77.52,81,1014,6.51,0
9,tamiahua,21.27,-97.45,78.51,88,1010,9.31,0


In [7]:
# Customize the size of the figure
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["latitude", "longitude"]]

# Assign the marker layer to a variable
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [8]:
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Name</dt><dd>{row.cities}</dd>
                <dt>Temp</dt><dd>{row.temperature}</dd>
                <dt>Cloudiness</dt><dd>{row.cloudiness}%</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [9]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

coordinates = ideal_cities[["latitude", "longitude"]]

marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
ideal_cities.head(1)

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness
0,antalaha,-14.9,50.28,72.91,83,1016,6.53,0


In [27]:
hotel_df = pd.DataFrame(df, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,NaN,NaN,NaN,NaN,
1,NaN,NaN,NaN,NaN,
2,NaN,NaN,NaN,NaN,
3,NaN,NaN,NaN,NaN,
4,NaN,NaN,NaN,NaN,
...,...,...,...,...,...
561,NaN,NaN,NaN,NaN,
562,NaN,NaN,NaN,NaN,
563,NaN,NaN,NaN,NaN,
564,NaN,NaN,NaN,NaN,


In [11]:
params = {"key": g_key}

base_url = "https://maps.googleapis.com/maps/api/geocode/json"

lat = ideal_cities["latitude"][1]
lng = ideal_cities["longitude"][1]

# update address key value
params['latlng'] = f"{lat},{lng}"

# make request
thing = requests.get(base_url, params=params)

thing = thing.json()
thing["results"][0]["address_components"]


for addressComp in thing["results"][0]["address_components"]:
    if addressComp["types"][0] == "country":
        print(addressComp["long_name"])

United States


In [12]:
def getCountryForCity(lat, long):    
    rtnCountry = ""
    
    params = {"key": g_key}

    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    lat = ideal_cities["latitude"][indx]
    lng = ideal_cities["longitude"][indx]

    # update address key value
    params['latlng'] = f"{lat},{lng}"

    # make request
    thing = requests.get(base_url, params=params)

    thing = thing.json()
    thing["results"][0]["address_components"]


    for addressComp in thing["results"][0]["address_components"]:
        if addressComp["types"][0] == "country":
            rtnCountry = addressComp["long_name"]
            
    return rtnCountry

In [13]:
countries = []
for indx, row in ideal_cities.iterrows():
    countries.append(getCountryForCity(row.latitude, row.longitude))

In [14]:
countries

['Madagascar',
 'United States',
 'The Bahamas',
 'United Arab Emirates',
 'Peru',
 'Angola',
 'Cape Verde',
 'Madagascar',
 'Brazil',
 'Mexico',
 'United States',
 'Afghanistan',
 'Egypt']

In [15]:
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

lat = ideal_cities["latitude"][0]
lng = ideal_cities["longitude"][0]

params["location"] = f"{lat},{lng}"

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

name_address = requests.get(base_url, params=params)
name_address = name_address.json()

name_address["results"][0]["name"]

'Villa-Malaza'

In [16]:
name_address["results"][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': -14.9061237, 'lng': 50.2785486},
  'viewport': {'northeast': {'lat': -14.90481452010728,
    'lng': 50.27988962989272},
   'southwest': {'lat': -14.90751417989272, 'lng': 50.27718997010728}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/lodging-71.png',
 'id': 'c869993e16cce59227e9b9e7aa89b468e03c7156',
 'name': 'Villa-Malaza',
 'photos': [{'height': 1536,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/102604115630443166232">A Google User</a>'],
   'photo_reference': 'CmRaAAAANrF1qfiK8yTi1AannXkiEUluo00GpUNVIISkIwzC-rrRzjty7gSKm9SfNpCXAVjB-9uTNljlvJkj6Bc6B702SrOplMKKhv3GoDwf816W4wRCkLLsfERF4BZeIJkvZSglEhBvzZbplTmtPHXfHuIT-nDtGhTrzpuZP-dRYcfirFXHEFNr2q3B2A',
   'width': 2560}],
 'place_id': 'ChIJ_____whsPSIRUD_-6qOm3e0',
 'plus_code': {'compound_code': '37VH+HC Antalaha, Madagascar',
  'global_code': '5HQG37VH+HC'},
 'rating': 0,
 'reference': 'ChIJ_____whsPSIRUD_-6qOm3e0',
 'sc

In [17]:
# Use the lat/lng we recovered to identify airports

names = []
addresses = []
ratings = []

params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotel",
    "key": g_key
}

for index, row in ideal_cities.iterrows():
    # get lat, lng from df
    lat = row["latitude"]
    lng = row["longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)

    # convert to json
    name_address = name_address.json()
    
    try:
        names.append(name_address["results"][0]["name"])
        addresses.append(name_address["results"][0]["vicinity"])
        ratings.append(name_address["results"][0]["rating"])
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        names.append("")
        addresses.append("")
        ratings.append("")

Missing field/result... skipping.
Missing field/result... skipping.


In [18]:
names

['Villa-Malaza',
 '',
 'Rock Sound Club',
 'Sheraton Abu Dhabi Hotel & Resort',
 'Hospedaje EMBAJADA',
 'iu Hotel Sumbe',
 'Hotel Xaguate',
 'Palissandre Cote Ouest resort & SPA',
 'Salinas do Maragogi All Inclusive Resort',
 'Mission Of The Sun Hotel',
 'Hampton Inn Key Largo, FL',
 '',
 'Sofitel Legend Old Cataract Aswan']

In [19]:
ideal_cities["country"] = countries
ideal_cities["hotels"] = names
ideal_cities["addresses"] = addresses
ideal_cities["ratings"] = ratings

ideal_cities

,cities,latitude,longitude,temperature,humidity,pressure,wind_speed,cloudiness,country,hotels,addresses,ratings
0,antalaha,-14.90,50.28,72.91,83,1016,6.53,0,Madagascar,Villa-Malaza,Antalaha,0
1,sumter,28.71,-82.08,72.70,78,1016,1.99,0,United States,,,
2,rock sound,24.90,-76.20,75.83,65,1016,8.90,0,The Bahamas,Rock Sound Club,Bahamas,3.8
3,abu dhabi,24.47,54.37,78.64,73,1010,1.12,0,United Arab Emirates,Sheraton Abu Dhabi Hotel & Resort,Corniche Rd,4.3
4,contamana,-7.33,-75.02,74.19,87,1010,1.12,0,Peru,Hospedaje EMBAJADA,"Pasaje Cornejo Portugal, Contamana",0
5,sumbe,-11.21,13.84,77.95,82,1011,5.73,0,Angola,iu Hotel Sumbe,"Estrada Nacional, Sumbe",3.4
6,sao filipe,14.90,-24.50,74.61,73,1014,6.49,0,Cape Verde,Hotel Xaguate,"Sao Filipe, Rua da Posto, Fogo",3.4
7,morondava,-20.28,44.28,72.97,67,1015,5.01,0,Madagascar,Palissandre Cote Ouest resort & SPA,Morondava,4.6
8,maragogi,-9.01,-35.22,77.52,81,1014,6.51,0,Brazil,Salinas do Maragogi All Inclusive Resort,"Rodovia AL-101 Norte Km 124, s/n - Sitio Carió...",4.7
9,tamiahua,21.27,-97.45,78.51,88,1010,9.31,0,Mexico,Mission Of The Sun Hotel,"Aquiles Serdan S/N, Zona Centro, Tamiahua",4.2


In [20]:
# Store latitude and longitude in locations
locations = df[["latitude", "longitude"]]

# Fill NaN values and convert to float
humid = df["humidity"]

In [21]:
city_info = []

for indx, row in ideal_cities.iterrows():
    info_box = f"""
                <dl>
                <dt>Hotel Name</dt><dd>{row.hotels}</dd>
                <dt>City</dt><dd>{row.cities}</dd>
                <dt>Country</dt><dd>{row.country}</dd>
                </dl>
                """
    
    city_info.append(info_box)

In [22]:
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}

# Plot Heatmap
fig = gmaps.figure(map_type="HYBRID", layout=figure_layout)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, dissipating=True)


# Add layer
fig.add_layer(heat_layer)

#add points
coordinates = ideal_cities[["latitude", "longitude"]]
marker_layer = gmaps.marker_layer(coordinates, info_box_content=city_info)
fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

In [28]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


InvalidPointException: nan is not a valid latitude. Latitudes must lie between -90 and 90.

In [29]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display figure


InvalidPointException: nan is not a valid latitude. Latitudes must lie between -90 and 90.